In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
from datetime import datetime, date

In [3]:
fire_risk_path='./data/Brandriskdata 2000-2020.csv'
reported_fires_path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'
#'PunktID': str, 'E': str, 'N': str,
rtypes={ 'PunktID': str}

In [4]:
#Reading csv file from path with dtypes
reported_fire_risk = pd.read_csv(fire_risk_path, sep=';', dtype=rtypes )
reported_fires = pd.read_excel(reported_fires_path)

In [5]:
risk_sample = reported_fire_risk
fires_sample = reported_fires

In [6]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       2
N                       2
Kommun                  2
Datum                   2
Temp                14617
Tmedel              38049
Nederbord           23520
RH                  33459
Vindhastighet        3991
Vindriktning     12310774
FFMC              1701172
DMC               1701172
DC                1701172
ISI               1701172
BUI               1701172
FWI               1701172
FWI_index         1701172
HBV_o             1404342
HBV_u             1404342
HBV               1404342
HBV_index         1404342
Gras              6855866
dtype: int64
Total rows: 
19188859


In [7]:
risk_sample[['E','N']] = risk_sample[['E','N']].fillna(0)
risk_sample['N'] = risk_sample['N'].astype(int)
risk_sample['E'] = risk_sample['E'].astype(int)

In [8]:
risk_sample.dropna(subset=['FWI'], how='all', inplace=True)
risk_sample.dropna(subset=['Tmedel'], how='all', inplace=True)


In [9]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  0
Datum                   0
Temp                    0
Tmedel                  0
Nederbord               0
RH                   3989
Vindhastighet           0
Vindriktning     11068726
FFMC                    0
DMC                     0
DC                      0
ISI                     0
BUI                     0
FWI                     0
FWI_index               0
HBV_o               39890
HBV_u               39890
HBV                 39890
HBV_index           39890
Gras              6777056
dtype: int64
Total rows: 
17474853


In [14]:
fireRisk = risk_sample.drop(columns=['Nederbord',
       'RH', 'Vindhastighet', 'Vindriktning', 'FFMC', 'DMC', 'DC', 'ISI',
       'BUI', 'HBV_o', 'HBV_u', 'HBV', 'HBV_index', 'Tmedel',
       'Gras','Temp','E','N'])
fireRisk

,PunktID,Kommun,Datum,FWI,FWI_index
0,1,2021.0,2000-04-01,0,1.0
1,1,2021.0,2000-04-02,0,1.0
2,1,2021.0,2000-04-03,0,1.0
3,1,2021.0,2000-04-04,0,1.0
4,1,2021.0,2000-04-05,0,1.0
...,...,...,...,...,...
19188852,7892,1737.0,2020-12-27,0,-1.0
19188853,7892,1737.0,2020-12-28,0,-1.0
19188854,7892,1737.0,2020-12-29,0,-1.0
19188855,7892,1737.0,2020-12-30,0,-1.0


In [52]:
def _convert_columns_totype(df,columns,type):
    for column in columns:
        df[column] = df[column].astype(type)
    return df

In [53]:
_convert_columns_totype(fireRisk,['FWI','Kommun'],float)

,PunktID,Kommun,Datum,FWI,FWI_index
0,1,2021.0,2000-04-01,0.0,1.0
1,1,2021.0,2000-04-02,0.0,1.0
2,1,2021.0,2000-04-03,0.0,1.0
3,1,2021.0,2000-04-04,0.0,1.0
4,1,2021.0,2000-04-05,0.0,1.0
...,...,...,...,...,...
19188852,7892,1737.0,2020-12-27,0.0,-1.0
19188853,7892,1737.0,2020-12-28,0.0,-1.0
19188854,7892,1737.0,2020-12-29,0.0,-1.0
19188855,7892,1737.0,2020-12-30,0.0,-1.0


In [15]:
#Values needs to be float to be able to do means, to convert to float we first change all ',' to '.'.
fireRisk['FWI'] = fireRisk['FWI'].str.replace(',','.')
fireRisk['FWI'] = fireRisk['FWI'].astype(float)
fireRisk['Kommun'] = fireRisk['Kommun'].astype(int)
#Datum needs to be in datetime format, here we use pd.to_datetime to convert the strings
fireRisk['Datum'] = pd.to_datetime(fireRisk['Datum'], format='%Y-%m-%d')

In [16]:
#Grouping by Kommun and date (YYYY-MM-DD) and converting rest of values to mean values.
test = fireRiskT.groupby(['Datum','Kommun']).mean()
test = test.reset_index()
test = test.rename(columns={"Datum": "datum", "Kommun":"kommun"})

In [46]:

#Actual merge happens here
merged = pd.merge(fires_sample, test, how='inner', on=['kommun','datum'])
#merged = merged.drop(columns='index')



In [47]:
#It might be interesting to look at the days where the fires of the municipalities FWI_index is above 4
above4 = merged[merged['FWI_index']>=4]
above4 = above4.drop(columns=['verksamhetText','tid','sweref99Ost','sweref99Norr'])


In [48]:
def _filter_rows_by_values(df, column, keep_values):
    #Replace ~ with - try if it works
    return df[df[column].isin(keep_values)]

In [49]:
#Filter cause to only include campfire and BBQ
above4 = _filter_rows_by_values(above4, 'BEJBbrandorsakText', ['Grillning eller lägereld'])
above4

,ar,datum,kommun,kommunKortNamn,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,FWI,FWI_index
1739,2000,2000-05-12,1285,Eslöv,Grillning eller lägereld,200,0,0,17.825000,4.000000
1783,2000,2000-05-12,862,Emmaboda,Grillning eller lägereld,0,100,0,18.785714,4.000000
1784,2000,2000-05-12,862,Emmaboda,Grillning eller lägereld,0,100,0,18.785714,4.000000
1884,2000,2000-05-13,1276,Klippan,Grillning eller lägereld,0,0,1,25.475000,5.000000
1885,2000,2000-05-13,2282,Kramfors,Grillning eller lägereld,0,0,75,21.460000,4.300000
...,...,...,...,...,...,...,...,...,...,...
85447,2020,2020-08-19,381,Enköping,Grillning eller lägereld,0,5,0,23.822222,4.666667
85483,2020,2020-08-20,180,Stockholm,Grillning eller lägereld,0,50,0,27.200000,5.000000
85495,2020,2020-08-21,1980,Västerås,Grillning eller lägereld,0,0,1,24.222222,4.888889
85568,2020,2020-08-24,980,Gotland,Grillning eller lägereld,0,300,0,26.536957,5.195652


In [50]:
above4 = above4.rename(columns={ "datum":"Date", "kommun":"Municipality_name","BEJBbrandorsakText":"Cause_of_fire","kommunKortNamn":"Municipality_name","ar":"Year"})

In [51]:
above4

,Year,Date,Municipality_name,Municipality_name,Cause_of_fire,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,FWI,FWI_index
1739,2000,2000-05-12,1285,Eslöv,Grillning eller lägereld,200,0,0,17.825000,4.000000
1783,2000,2000-05-12,862,Emmaboda,Grillning eller lägereld,0,100,0,18.785714,4.000000
1784,2000,2000-05-12,862,Emmaboda,Grillning eller lägereld,0,100,0,18.785714,4.000000
1884,2000,2000-05-13,1276,Klippan,Grillning eller lägereld,0,0,1,25.475000,5.000000
1885,2000,2000-05-13,2282,Kramfors,Grillning eller lägereld,0,0,75,21.460000,4.300000
...,...,...,...,...,...,...,...,...,...,...
85447,2020,2020-08-19,381,Enköping,Grillning eller lägereld,0,5,0,23.822222,4.666667
85483,2020,2020-08-20,180,Stockholm,Grillning eller lägereld,0,50,0,27.200000,5.000000
85495,2020,2020-08-21,1980,Västerås,Grillning eller lägereld,0,0,1,24.222222,4.888889
85568,2020,2020-08-24,980,Gotland,Grillning eller lägereld,0,300,0,26.536957,5.195652
